In [1]:
# # The process
# 0. Learn about `Observable`s
# 1. Initialize simulation in a stepping manner
# 2. Initialize the `Observable`s of the animation 
# 3. Plot the `Observable`s and any other static elements
# 4. Create the "animation stepping function"
# 5. Test it
# 6. Save animations to videos
# 7. Interactive application

# Let's start!
# Load packages that will be used
using DynamicalSystems, OrdinaryDiffEq, GLMakie
using DataStructures: CircularBuffer


In [2]:
# %% 0. Learn about `Observable`s
# An `Observable` is a mutable container of an object of type `T`.
# `T` can be any type. The value of an `Observable` can then be 
# changed on the spot, just like updating any mutable container.
# (This is similar to the base structure `Ref`, if you're familiar)

# The important part here is that `Observable`s can be "listened" to.
# What does this mean...?

o = Observable(1) # Observable with values of type `Int`

l1 = on(o) do val # Create a listener `l1` of observable.
    println("Observable now has value $val")
end
# `l1` is triggered each time the value of `o` is updated.
# (demo in REPL, set `o[] = 2`.)

ObserverFunction defined at /home/czc/projects/working/stock/binance_gui/test.ipynb:13 operating on Observable{Int64} with 1 listeners. Value:
1

In [3]:
o[] = 2
o[] = 1

Observable now has value 2
Observable now has value 1


1

In [25]:
# We care about `Observable`s because Makie.jl is hooked up
# to this "listener" system. If any plotted element is
# initialized as an observable, then Makie.jl understands this.
# Updating the observable would update the plot values.

# For example:
ox = 1:4
oy = Observable(rand(4))
lw = Observable(2)


Observable{Int64} with 0 listeners. Value:
2

In [33]:
linesegs = []
for i in 1:4
    push!(linesegs, Point2f(i, rand()))
    push!(linesegs, Point2f(i, rand()))
end

linesegs1 = Observable(Point2f.(linesegs))
fig = Figure()
ax = Axis(fig[1, 1])
linesegments!(ax, linesegs1, color=:blue)
fig

In [30]:

# fig = Figure()
# ax = Axis(fig[1, 1])

tt = 1
while tt < 10
    # empty!(ax)
    linesegs = []
    for i in 1:4
        push!(linesegs, Point2f(i, rand()))
        push!(linesegs, Point2f(i, rand()))
    end
    linesegs1[] = Point2f.(linesegs)
    sleep(0.5)
    tt += 1
end

In [6]:
lw[] = 10
oy[] = rand(4)

4-element Vector{Float64}:
 0.8377526137057764
 0.8949079816320731
 0.3699002558187504
 0.08459686094524677

In [7]:

# This simple process is the basis of creating 
# both animations, as well as interactive apps with Makie.jl.
# So in the following let's begin making a simple visualization
# and interactive application of the double pendulum


In [8]:
# %% 1. Initialize simulation in a stepping manner
# (this can also be done with a pre-run simulation)
# the goal is to create a "step" function which
# once called it progresses the data for one animation frame
const L1 = 1.0
const L2 = 0.9
M = 2
u0 = [π/3, 0, 3π/4, -2]
dp = Systems.double_pendulum(u0; L1, L2)

# Solve diffeq with constant step for smoother curves
diffeq = (alg = Tsit5(), adaptive = false, dt = 0.005)

integ = integrator(dp, u0; diffeq)

function xycoords(state)
    θ1 = state[1]
    θ2 = state[3]
    x1 = L1 * sin(θ1)
    y1 = -L1 * cos(θ1)
    x2 = x1 + L2 * sin(θ2)
    y2 = y1 - L2 * cos(θ2)
    return x1,x2,y1,y2
end

# `integ` is an integrator. `step!(integ)` progresses the integrator
# for one step. `integ.u` is the system state at current step. 
# Then `xycoords` converts the states of the integrator
# to their plottable format. So we can imagine something like 
function progress_for_one_step!(integ)
    step!(integ)
    u = integ.u
    return xycoords(u)
end
# to be our stepping function that returns the new data.

# If we had finite data instead of a forever-running animation, 
# then the "stepping function" would simply be to progress the index `i`
# of the existing data one step forwards...


progress_for_one_step! (generic function with 1 method)

In [9]:
# %% 2. Initialize the `Observable`s of the animation 
# You need to think of this in advance: what things will to be 
# animated, and what other plotting elements will be static? 
# Animated elements will need to become `Observable`s.

# Here the animated elements will be: balls and rods making the
# double pendulum, and the tail (trajectory) of the pendulum.
x1,x2,y1,y2 = xycoords(u0)
rod   = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])
# (Remember: the most optimal way to plot 2D things in Makie.jl is to
# give it a vector of `Point2f`, the coordinates for the plot)

# Here we have initialized two _different_ observables, because
# rods and balls will be plotted in a different manner (lines/scatter)

# Next is the observable for the tail
tail = 300 # length of plotted trajectory, in units of `dt`
# The circular buffer datastructure makes making stepping-based
# animations very intuitive
traj = CircularBuffer{Point2f}(tail)
fill!(traj, Point2f(x2, y2)) # add correct values to the circular buffer
traj = Observable(traj) # make it an observable


Observable{CircularBuffer{Point{2, Float32}}} with 0 listeners. Value:
Point{2, Float32}[[1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611]  …  [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611], [1.5024215, 0.13639611]]

In [10]:
# %% 3. Plot the `Observable`s and any other static elements
# Before plotting we need to initialie a figure
fig = Figure(); display(fig)


GLMakie.Screen(...)

In [11]:
# in my experience it leads to cleaner code if we first initialize 
# an axis and populate it accordingly.
ax = Axis(fig[1,1])


Axis with 1 plots:
 ┗━ Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleP{3, Float32, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}}, GeometryBasics.FaceView{GeometryBasics.TriangleP{3, Float32, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}}, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, StructArrays.StructVector{GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}, NamedTuple{(:position, :normals), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}}, Int64}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}}


In [12]:

# Now we plot the observables _directly_! First the pendulum
lines!(ax, rod; linewidth = 4, color = :purple)
scatter!(ax, balls; marker = :circle, strokewidth = 2, 
    strokecolor = :purple,
    color = :black, markersize = [8, 12]
)


Scatter{Tuple{Vector{Point{2, Float32}}}}

In [13]:

# then its trajectory, with a nice fadeout color
c = to_color(:purple)
tailcol = [RGBAf(c.r, c.g, c.b, (i/tail)^2) for i in 1:tail]
lines!(ax, traj; linewidth = 3, color = tailcol)


Lines{Tuple{CircularBuffer{Point{2, Float32}}}}

In [14]:

# We can also plot now any other static elements
ax.title = "double pendulum"
ax.aspect = DataAspect()
l = 1.05(L1+L2)
xlims!(ax, -l, l)
ylims!(ax, -l, 0.5l)


In [15]:

# %% 4. Create the "animation stepping function"
# Using the functions of step 1, we now define a function
# that updates the observables. Makie.jl understands observable
# updates and directly reflects this on the plotted elements.
function animstep!(integ, rod, balls, traj)
    x1,x2,y1,y2 = progress_for_one_step!(integ)
    rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
    balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
    push!(traj[], Point2f(x2, y2))
    traj[] = traj[] # <- important! Updating in-place the value of an
                    # `Observable` does not trigger an update!
end


animstep! (generic function with 1 method)

In [24]:
animstep!(integ, rod, balls, traj);

In [16]:

# %% 5. Test it
for i in 1:1000
    animstep!(integ, rod, balls, traj)
    sleep(0.01)
end


In [17]:

# cool it works. Let's wrap up the creation of the observables
# and plots in a function (just to re-initialie everything)
function makefig(u0)
    dp = Systems.double_pendulum(u0; L1, L2)
    integ = integrator(dp, u0; diffeq...)
    x1,x2,y1,y2 = xycoords(u0)
    rod   = Observable([Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)])
    balls = Observable([Point2f(x1, y1), Point2f(x2, y2)])
    traj = CircularBuffer{Point2f}(tail)
    fill!(traj, Point2f(x2, y2)) # add correct values to the circular buffer
    traj = Observable(traj) # make it an observable
    fig = Figure(); display(fig)
    ax = Axis(fig[1,1])
    lines!(ax, rod; linewidth = 4, color = :purple)
    scatter!(ax, balls; marker = :circle, strokewidth = 2, 
        strokecolor = :purple,
        color = :black, markersize = [8, 12]
    )
    lines!(ax, traj; linewidth = 3, color = tailcol)
    ax.title = "double pendulum"
    ax.aspect = DataAspect()
    l = 1.05(L1+L2)
    xlims!(ax, -l, l)
    ylims!(ax, -l, 0.5l)
    # also return the figure object, we'll ues it!
    return fig, integ, rod, balls, traj
end
    


makefig (generic function with 1 method)

In [34]:
# %% 6. Save animations to videos
fig, integ, rod, balls, traj = makefig(u0)
frames = 1:200
record(fig, "video.mp4", frames; framerate = 60) do i # i = frame number
    for j in 1:5 # step 5 times per frame
        animstep!(integ, rod, balls, traj)
    end
    # any other manipulation of the figure here...
end # for each step of this loop, a frame is recorded


┌ Warning: Direct propagation of keyword arguments to DifferentialEquations.jl is deprecated.
│ From now on pass any DiffEq-related keywords as a `NamedTuple` using the
│ explicit keyword `diffeq` instead.
└ @ DynamicalSystemsBase /home/czc/.julia/packages/DynamicalSystemsBase/nNJRo/src/core/continuous.jl:48


"video.mp4"

In [44]:
# %% 7. Interactive application
# Makie.jl has tremendously strong capabilities for real-time
# interactivity. To learn all of this takes time of course,
# and you'll need to consult the online documentation.
# Here we will do two interactions: 1) a play/stop button
# 2) clicking on the screen and getting a new initial condition!

fig, integ, rod, balls, traj = makefig(u0)
# The run button is actually pretty simple, we'll add it below the plot
run = Button(fig[2, 1]; label="run", tellwidth=false)
# This button will start/stop an animation. It's actually surprisingly
# simple to do this. The magic code is:
isrunning = Observable(false)
on(run.clicks) do clicks
    isrunning[] = !isrunning[]
end
on(run.clicks) do clicks
    @async while isrunning[]
        isopen(fig.scene) || break # ensures computations stop if closed window
        animstep!(integ, rod, balls, traj)
        sleep(0.001) # or `yield()` instead
    end
end

# `on` an important Observables function when one starts
# doing advanced stuff. It triggers a piece of code once an observable
# triggers its update.

# We'll add one more interactive feature which will trigger once
# we click on the axis. Notice that by default makie performs a zoom
# once one clicks on the axis, so we'll disable this
ax = content(fig[1, 1])
Makie.deactivate_interaction!(ax, :rectanglezoom)
# and we'll add a new trigger using the `select_point` function:
spoint = select_point(ax.scene)

# Now we see that we can click on the screen and the `spoint` updates!

# Okay, let's do something useful when it triggers
function θωcoords(x, y)
    θ = atan(y, x) + π / 2
    return SVector(θ, 0, 0, 0)
end

on(spoint) do z
    x, y = z
    @show x, y
    plot!(z)
    u = θωcoords(x, y)
    # reinit!(integ, u)
    # # Reset tail and balls to new coordinates
    # x1,x2,y1,y2 = xycoords(u)
    # traj[] .= fill(Point2f(x2, y2), length(traj[]))
    # traj[] = traj[]
    # rod[] = [Point2f(0, 0), Point2f(x1, y1), Point2f(x2, y2)]
    # balls[] = [Point2f(x1, y1), Point2f(x2, y2)]
end

# And that's the end of the tutorial!

┌ Warning: Direct propagation of keyword arguments to DifferentialEquations.jl is deprecated.
│ From now on pass any DiffEq-related keywords as a `NamedTuple` using the
│ explicit keyword `diffeq` instead.
└ @ DynamicalSystemsBase /home/czc/.julia/packages/DynamicalSystemsBase/nNJRo/src/core/continuous.jl:48


ObserverFunction defined at /home/czc/projects/working/stock/binance_gui/test.ipynb:46 operating on Observable{Point{2, Float32}} with 1 listeners. Value:
Float32[0.0, 0.0]

In [5]:
using GLMakie
fig = Figure()

menu = Menu(fig, options = ["viridis", "heat", "blues"])

funcs = [sqrt, x->x^2, sin, cos]

menu2 = Menu(fig, options = zip(["Square Root", "Square", "Sine", "Cosine"], funcs))

ax = Axis(fig[1, 1], ygridcolor="#65866b",
    xgridcolor="#65866b", xgridstyle=:dash, ygridstyle=:dash)

func = Observable{Any}(funcs[1])

ys = lift(func) do f
    f.(0:0.3:10)
end
scat = scatter!(ax, ys, markersize = 10px, color = ys)

cb = Colorbar(fig[1, 2], scat)

fig[1, 3] = vgrid!(
    Label(fig, "Colormap", width = nothing),
    menu,
    Label(fig, "Function", width = nothing),
    menu2;
    tellheight = false, width = 200)

on(menu.selection) do s
    scat.colormap = s
end

on(menu2.selection) do s
    func[] = s
    autolimits!(ax)
end

# menu2.is_open = true

fig



ErrorException: Metadata array needs to have same length as data.
                    Found 4 data items, and 12 metadata items

In [8]:
fig = Figure(); display(fig);

leftTopPane = fig[1, 1] = GridLayout()
leftBottomPane = fig[2, 1] = GridLayout()
rightPane = fig[1:2, 2] = GridLayout()

ax1 = Axis(leftTopPane[1, 1], ygridcolor="#65866b",
xgridcolor="#65866b", xgridstyle=:dash, ygridstyle=:dash)
ax2 = Axis(leftBottomPane[1, 1])

# sld = Slider(leftBottomPane[2,1],range=-10:1:10,startvalue=0)

btnOpen = Button(rightPane[1,1],label="Open a file...")
btnDo = Button(rightPane[3,1], label="Do Something!")
btnDoMore = Button(rightPane[4,1],label="Do Something Else!")

txtTest = Textbox(rightPane[2,1],stored_string="This is Text!",tellwidth=false)

Textbox()

In [12]:

tt = 2
while tt < 10
    fig = Figure()
    display(fig)
    ax1 = Axis(fig[1, 1], ygridcolor="#65866b",
        xgridcolor="#65866b", xgridstyle=:dash, ygridstyle=:dash)
    linesegments!(ax1, [3, 4])
    fig
    tt += 1
end

In [14]:
a = Observable(Axis(fig[1,1], ygridcolor="#65866b",
xgridcolor="#65866b", xgridstyle=:dash, ygridstyle=:dash))

Observable{Axis} with 0 listeners. Value:
Axis (1 plots)

In [23]:
linesegments!(a[], [3, 8])
fig

In [2]:
using GLMakie
using Makie
using GLMakie: GLFW
using Makie.ColorTypes: RGBA, N0f8
# TODO do not leak memory
cursor1 = GLFW.CreateStandardCursor(GLFW.CROSSHAIR_CURSOR)

img = fill(RGBA{N0f8}(0,0,0,0), 15, 15)
img[8, :] .= RGBA(1.0, 0.0, 1.0, 1.0)
img[:, 8] .= RGBA(1.0, 0.0, 1.0, 1.0)
# img[1, 1] is upper left
# img[1, end] is upper right

img_ = collect(reinterpret(NTuple{4, UInt8}, img))
hotspot = (8, 8)
cursor2 = ccall((:glfwCreateCursor, GLFW.libglfw), GLFW.Cursor, (Ref{GLFW.GLFWImage}, Cint, Cint), img_, hotspot[1], hotspot[2])

fig = Figure()
ax = Axis(fig[1, 1], title = "Demo")
ax.autolimitaspect = 1.0
scatter!(ax, [(-5, -5), (5, 5)])
screen = display(fig)
window = GLMakie.to_native(screen)

cursor_Fdata_obs = lift(events(fig).mouseposition) do mp
    # `mp` is relative to the window, throw it away.
    return mouseposition(ax.scene)
end

on(cursor_Fdata_obs) do cursor_Fdata
    cursor_Fraster = Tuple(round.(Int, cursor_Fdata))
    if iseven(sum(cursor_Fraster)) # checkboard pattern
        GLFW.SetCursor(window, cursor1)
    else
        GLFW.SetCursor(window, cursor2)
    end
end

ObserverFunction defined at /home/czc/projects/working/stock/binance_gui/test.ipynb:31 operating on Observable{Point{2, Float32}} with 1 listeners. Value:
Float32[-5.9888887, -6.349445]

In [150]:
using GLMakie

data = rand(5)

fig = Figure()
ax1 = Axis(fig[1, 1], yaxisposition=:right)
deactivate_interaction!(ax1, :rectanglezoom)
lines!(ax1, data)
mouse_pos = lift(events(ax1).mouseposition) do mp
    mouseposition(ax1.scene)
end

mouse_pos_x = @lift $mouse_pos[1]
mouse_pos_y = @lift $mouse_pos[2]
vlines!(ax1, mouse_pos_x, color=:red, linestyle=:dash, linewidth=1)
hlines!(ax1, mouse_pos_y, color=:red, linestyle=:dash, linewidth=1)

ylims!(ax1, 0, 1)
xlims!(ax1, 0, 6)

mouse_pos_click = Observable([0.0, 0.0])
mouse_pos_click_y = @lift $mouse_pos_click[2]
mouse_pos_click_y_str = @lift string(round($mouse_pos_click_y, sigdigits=3))

buy = hlines!(ax1, mouse_pos_click_y, color=:green, linestyle=:dash, linewidth=1)
buy_text = text!(5, mouse_pos_click_y, text=mouse_pos_click_y_str)

on(events(ax1).mousebutton) do event
    if event.button == Mouse.left
        if event.action == Mouse.press
            mouse_pos_click[] = mouseposition(ax1.scene)
        end
    end
    return Consume(false)
end

tt = Observable("nothing")

on(events(ax1).keyboardbutton) do event
    if ispressed(fig, (Keyboard.left_control, Keyboard.l))
        tt[] = "long"
    elseif ispressed(fig, (Keyboard.left_control, Keyboard.s))
        tt[] = "short"
    end
end

fig

In [156]:
events(ax1).keyboardstate

Set{Makie.Keyboard.Button}()

In [147]:
t3

ObserverFunction defined at /home/czc/projects/working/stock/binance_gui/test.ipynb:39 operating on Observable{Makie.KeyEvent} with 3 listeners. Value:
Makie.KeyEvent(Makie.Keyboard.l, Makie.Keyboard.release)

In [91]:
delete!(ax1, buy)

Axis with 5 plots:
 ┣━ Mesh{Tuple{GeometryBasics.Mesh{3, Float32, GeometryBasics.TriangleP{3, Float32, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}}, GeometryBasics.FaceView{GeometryBasics.TriangleP{3, Float32, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}}, GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}, GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}, StructArrays.StructVector{GeometryBasics.PointMeta{3, Float32, Point{3, Float32}, (:normals,), Tuple{Vec{3, Float32}}}, NamedTuple{(:position, :normals), Tuple{Vector{Point{3, Float32}}, Vector{Vec{3, Float32}}}}, Int64}, Vector{GeometryBasics.NgonFace{3, GeometryBasics.OffsetInteger{-1, UInt32}}}}}}}
 ┣━ Lines{Tuple{Vector{Point{2, Float32}}}}
 ┣━ Combined{Makie.vlines, Tuple{Float32}}
 ┣━ Combined{Makie.hlines, Tuple{Float32}}
 ┗━ MakieCore.Text{Tuple{Vector{Point{2, Float32}

In [129]:
using GLMakie

colors = to_colormap(:cyclic_mrybm_35_75_c68_n256)
idx = Observable(1)
color = map(i -> colors[mod1(i, length(colors))], idx)
points = Observable(Point2f[])

scene = Scene(camera = campixel!)
linesegments!(scene, points, color = color)
scatter!(scene, points, color = :gray, strokecolor = color, strokewidth = 1)

on(events(scene).mousebutton) do event
    if event.button == Mouse.left && event.action == Mouse.press
        mp = events(scene).mouseposition[]
        push!(points[], mp, mp)
        notify(points)
    end
end

on(events(scene).mouseposition) do mp
    mb = events(scene).mousebutton[]
    if mb.button == Mouse.left && (mb.action == Mouse.press || mb.action == Mouse.repeat)
        points[][end] = mp
        notify(points)
    end
end

on(events(scene).scroll) do (dx, dy)
    idx[] = idx[] + sign(dy)
end

t4 = on(events(scene).keyboardbutton) do event
    if event.action == Keyboard.press || event.action == Keyboard.repeat
        length(points[]) > 1 || return nothing
        if event.key == Keyboard.backspace
            pop!(points[])
            pop!(points[])
            notify(points)
        elseif event.key == Keyboard.delete
            popfirst!(points[])
            popfirst!(points[])
            notify(points)
        end
    end
    event.key
end

scene

In [132]:
t4== Keyboard.backspace

false

In [93]:
delete!(ax1, buy)
delete!(ax1, buy_text)

ErrorException: Combined{Makie.hlines, Tuple{Float64}} not in scene!

In [65]:
mouse_pos

Observable{Point{2, Float32}} with 2 listeners. Value:
Float32[-0.88116574, 0.5840164]

In [20]:

function indicator(ax::Axis,ob)
    register_interaction!(ax, :indicator) do event::MouseEvent, axis
    if event.type === MouseEventTypes.over
        ob[] = event.data
    end
    end
end
function indicator(grid::GridLayout,ob)
    foreach(Axis,grid;recursive=true) do ax
    indicator(ax,ob)
    end
end
function indicator(grid::GridLayout)
    ob = Observable(Point2f0(0.,0.))
    indicator(grid,ob)
    ob
end
function indicator(fig,args...; tellwidth=false, kwargs...)
    Label(
        fig,
        lift(ind->"x: $(ind[1])  y: $(ind[2])",indicator(fig.layout)),
        args...; tellwidth=tellwidth, kwargs...
    )
end

plot_range_1 = range(0,1;length=1000)
fig = Figure()
ax1 = fig[1,1] = Axis(fig)
l1 = lines!(ax1,plot_range_1,plot_range_1 .^ 2)
txt= fig[2,:] = indicator(fig)
display(fig)



GLMakie.Screen(...)

In [157]:
cfg = Observable(Dict("a"=>1))

Observable{Dict{String, Int64}} with 0 listeners. Value:
Dict("a" => 1)

15

In [161]:
cfg[]["b"]=2

2

In [162]:
cfg[]

Dict{String, Int64} with 2 entries:
  "b" => 2
  "a" => 1

In [175]:
t5 = @lift ($cfg)["a"]

Observable{Int64} with 0 listeners. Value:
16

In [173]:
cfg[]["a"]=16
t5

Observable{Int64} with 0 listeners. Value:
15